In [43]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon, rankdata
import os

In [47]:
df = pd.read_excel(FILE, sheet_name=SHEET, header=0)

# Drop completely empty rows
df = df.dropna(how="all")

# If first column is Participant ID, ensure it's numeric and drop invalid rows
df = df[df.iloc[:, 0].notna()]

# Convert "Level X" → integer
def level_to_int(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s.lower().startswith("level"):
        return int(s.split()[-1])
    return int(float(s))

# Apply conversion to all performance columns (skip Participant column)
for col in df.columns[1:]:
    df[col] = df[col].apply(level_to_int)

# Confirm correct N
print("Number of participants detected:", len(df))
df.tail()

Number of participants detected: 45


,Participant,Info & Data Literacy (Infinity),Info & Data Literacy (DiGiUP),Safety (Infinity),Safety (DiGiUP),Problem Solving (Infinity),Problem Solving (DiGiUP)
40,41,3,3,3,3,1,3
41,42,1,3,3,3,1,2
42,43,2,3,1,2,3,3
43,44,1,1,2,3,1,2
44,45,2,3,2,3,1,2


In [48]:
# ---- Identify paired columns (pre/post per area) ----
# Expected names like:
# "Info & Data Literacy (Infinity)" , "Info & Data Literacy (DiGiUP)" etc.
AREAS = ["Info & Data Literacy", "Safety", "Problem Solving"]
PRE_SUFFIX = "(Infinity)"
POST_SUFFIX = "(DiGiUP)"

In [49]:
pairs = []
for area in AREAS:
    pre_col = f"{area} {PRE_SUFFIX}"
    post_col = f"{area} {POST_SUFFIX}"
    if pre_col not in df.columns or post_col not in df.columns:
        # Try alternative naming (without a space before parentheses)
        pre_col = f"{area}{PRE_SUFFIX}"
        post_col = f"{area}{POST_SUFFIX}"
    pairs.append((area, pre_col, post_col))

In [50]:
# ---- Wilcoxon Z computation (with tie correction on |d| ranks) ----
def wilcoxon_z(pre, post, continuity=True):
    """
    Returns:
      n_used, W_plus, W_minus, T (min(W+,W-)), z, r
    Notes:
      - Removes zero differences (standard Wilcoxon signed-rank practice)
      - Uses average ranks for ties in |d|
      - Variance tie correction: var(W+) = (n(n+1)(2n+1) - sum(t^3 - t))/24
    """
    pre = np.asarray(pre, dtype=float)
    post = np.asarray(post, dtype=float)

    d = post - pre
    d = d[~np.isnan(d)]
    d_nz = d[d != 0]  # drop zeros
    n = len(d_nz)
    if n == 0:
        return dict(n_used=0, W_plus=np.nan, W_minus=np.nan, T=np.nan, z=np.nan, r=np.nan)

    abs_d = np.abs(d_nz)

    # Average ranks for ties in |d|
    ranks = rankdata(abs_d, method="average")

    W_plus = float(np.sum(ranks[d_nz > 0]))
    W_minus = float(np.sum(ranks[d_nz < 0]))
    T = min(W_plus, W_minus)

    # Mean/variance of W_plus under H0
    mean_W = n * (n + 1) / 4.0

    # Tie correction based on multiplicities in abs_d
    _, counts = np.unique(abs_d, return_counts=True)
    tie_term = np.sum(counts**3 - counts)  # sum(t^3 - t)

    var_W = (n * (n + 1) * (2 * n + 1) - tie_term) / 24.0
    sd_W = np.sqrt(var_W) if var_W > 0 else np.nan

    # z for W_plus (normal approximation)
    if np.isnan(sd_W) or sd_W == 0:
        z = np.nan
    else:
        cc = 0.5 if continuity else 0.0
        # Continuity correction direction depends on whether W_plus is above/below mean
        z = (W_plus - mean_W - np.sign(W_plus - mean_W) * cc) / sd_W

    r = np.abs(z) / np.sqrt(n) if (not np.isnan(z) and n > 0) else np.nan

    return dict(n_used=n, W_plus=W_plus, W_minus=W_minus, T=T, z=z, r=r)


In [51]:
# ---- Run analysis per area ----
rows = []
for area, pre_col, post_col in pairs:
    pre = df[pre_col].map(level_to_int)
    post = df[post_col].map(level_to_int)

    # Descriptives of change
    diff = post - pre
    n_total = diff.notna().sum()
    n_improve = int((diff > 0).sum())
    n_same = int((diff == 0).sum())
    n_worse = int((diff < 0).sum())
    median_pre = float(np.nanmedian(pre))
    median_post = float(np.nanmedian(post))
    median_diff = float(np.nanmedian(diff))

    # Wilcoxon from SciPy (paired)
    # 'pratt' keeps zeros in ranking adjustment; 'wilcox' drops zeros.
    # We typically report 'wilcox' for classic Wilcoxon signed-rank.
    sci = wilcoxon(pre, post, zero_method="wilcox", alternative="two-sided", method="auto")
    W_scipy = float(sci.statistic)
    p_value = float(sci.pvalue)

    # Compute W+, W-, Z, r ourselves (to report Z and r explicitly)
    wz = wilcoxon_z(pre, post, continuity=True)

    rows.append({
        "Area": area,
        "N_total": n_total,
        "Improved": n_improve,
        "Same": n_same,
        "Worse": n_worse,
        "Median_pre": median_pre,
        "Median_post": median_post,
        "Median_diff": median_diff,
        "Wilcoxon_W (SciPy)": W_scipy,
        "p_value": p_value,
        "n_used (non-zero diffs)": wz["n_used"],
        "W_plus": wz["W_plus"],
        "W_minus": wz["W_minus"],
        "Z (normal approx, cc)": wz["z"],
        "Effect size r": wz["r"],
    })

results = pd.DataFrame(rows)

In [52]:
# ---- Pretty display ----
pd.set_option("display.precision", 4)
results

,Area,N_total,Improved,Same,Worse,Median_pre,Median_post,Median_diff,Wilcoxon_W (SciPy),p_value,n_used (non-zero diffs),W_plus,W_minus,"Z (normal approx, cc)",Effect size r
0,Info & Data Literacy,45,32,13,0,1.0,2.0,1.0,0.0,1.8310e-07,32,528.0,0.0,5.5379,0.9790
1,Safety,45,28,17,0,2.0,3.0,1.0,0.0,3.9730e-07,28,406.0,0.0,5.6656,1.0707
2,Problem Solving,45,30,15,0,1.0,2.0,1.0,0.0,1.9618e-07,30,465.0,0.0,5.7464,1.0491
